In [2]:
import numpy as np
from scipy.signal.windows import cosine
import sys
import os
import re
import shutil
import glob # gogabgalab
from scipy.ndimage import uniform_filter
import noise
import pickle
from qdyn import qdyn

/opt/cray/pe/python/3.9.13.1/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [ ]:
current_working_directory = os.getcwd()

xoffset = int(re.search(r'_(\d+)x_', current_working_directory).group(1))
yoffset = int(re.search(r'_(\d+)y_', current_working_directory).group(1))
zoffset = int(re.search(r'_(\d+)z', current_working_directory).group(1))

print('x offset =', xoffset,'m, y offset =', yoffset,'m, z offset =', zoffset,'m')

In [1]:
"""
OPEN OUTPUT FILES
"""
# Instantiate the QDYN class object
p = qdyn()

for file in os.listdir():

    if file.startswith("meshdict.pkl"):
        # open file with dictionary
        dict_to_read = open(file, "rb")

        # assign dictionary to p.mesh_dict
        p.mesh_dict = pickle.load(dict_to_read)
    
    if file.startswith("setdict.pkl"):
        # open file with dictionary
        dict_to_read = open(file, "rb")

        # assign dictionary to p.mesh_dict
        p.set_dict = pickle.load(dict_to_read)
        
# read output
p.read_output(read_fault=True, read_ot=True, read_ox=True)

NameError: name 'os' is not defined

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
# Get the coordinates of the elements for the output time series

Nw = p.set_dict["NW"]
Nx = p.set_dict["NX"]
split = Nw // 2


i1 = p.set_dict["NW"]//4*p.set_dict["NX"] - p.set_dict["NX"]//2 # index of element in the center of Fault 1
i2 = p.set_dict["NW"]*3//4*p.set_dict["NX"] - p.set_dict["NX"]//2 # index of element in the center of Fault 1
x1 = p.mesh_dict["X"][i1] * 1e-3    
y1 = p.mesh_dict["Y"][i1] * 1e-3
z1 = p.mesh_dict["Z"][i1] * 1e-3

x2 = p.mesh_dict["X"][i2] * 1e-3
y2 = p.mesh_dict["Y"][i2] * 1e-3
z2 = p.mesh_dict["Z"][i2] * 1e-3

# Get the mesh values and reshape them into matrices
x = p.mesh_dict["X"].reshape((Nw, Nx)) * 1e-3
y = p.mesh_dict["Y"].reshape((Nw, Nx)) * 1e-3
z = p.mesh_dict["Z"].reshape((Nw, Nx)) * 1e-3
b = (p.mesh_dict["B"]).reshape((Nw, Nx))
aminb = (p.mesh_dict["A"] - p.mesh_dict["B"]).reshape((Nw, Nx))
adivb = (p.mesh_dict["A"] / p.mesh_dict["B"]).reshape((Nw, Nx))
dw = p.mesh_dict["DW"].reshape((Nw, Nx))
idx = np.arange(0,len(p.mesh_dict["X"])).reshape((Nw, Nx)) # indices

# Define colour scales for (a-b) and mesh spacing and indices
vmin = 0
vmax = abs(b.max())
norm = cm.colors.Normalize(vmin=vmin, vmax=vmax)
cmap = cm.BuPu


vmin2 = -max(abs(aminb.max()),abs(aminb.min()))
vmax2 = max(abs(aminb.max()),abs(aminb.min()))
#vmin2 = -0.0019
#vmax2 = 0.0019
norm2 = cm.colors.Normalize(vmin=vmin2, vmax=vmax2)
cmap2 = cm.coolwarm


vmin3 = adivb.min()
vmax3 = adivb.max()
norm3 = cm.colors.Normalize(vmin=vmin3, vmax=vmax3)
cmap3 = cm.Oranges

vmin4 = 0
vmax4 = abs(idx.max())
norm4 = cm.colors.Normalize(vmin=vmin4, vmax=vmax4)
cmap4 = cm.viridis


plt.close("all")
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(7, 7), subplot_kw={"projection": "3d"})


# Plot b
ax1 = axes[0,0]
CS = ax1.plot_surface(x[:split], y[:split], z[:split], facecolors=cmap(norm(b[:split])), antialiased=True, shade=False, zorder=2.5)
CS2 = ax1.plot_surface(x[split:], y[split:], z[split:], facecolors=cmap(norm(b[split:])), antialiased=True, shade=False)
cb1 = fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), ax=ax1, shrink=0.5, orientation="horizontal", pad=-0.05)
cb1.set_label("b")
ax1.set_title("b")

# Plot (a-b)
ax2 = axes[0,1]
CS3 = ax2.plot_surface(x[:split], y[:split], z[:split], facecolors=cmap2(norm2(aminb[:split])), antialiased=True, shade=False, zorder=2.5)
CS4 = ax2.plot_surface(x[split:], y[split:], z[split:], facecolors=cmap2(norm2(aminb[split:])), antialiased=True, shade=False)
cb2 = fig.colorbar(cm.ScalarMappable(norm=norm2, cmap=cmap2), ax=ax2, shrink=0.5, orientation="horizontal", pad=-0.05)
cb2.set_label("(a-b)")
ax2.set_title("(a-b) values")

# Plot (a/b)
ax3 = axes[1,0]
CS6 = ax3.plot_surface(x[split:], y[split:], z[split:], facecolors=cmap3(norm3(adivb[split:])), antialiased=True, shade=False, zorder=2.5)
CS5 = ax3.plot_surface(x[:split], y[:split], z[:split], facecolors=cmap3(norm3(adivb[:split])), antialiased=True, shade=False)

cb2 = fig.colorbar(cm.ScalarMappable(norm=norm3, cmap=cmap3), ax=ax3, shrink=0.5, orientation="horizontal", pad=-0.05)
cb2.set_label("a/b")
ax3.set_title("a/b")


# Plot mesh spacing (note that this is a binary distribution) + elements of output series
ax4 = axes[1,1]
P1 = ax4.scatter(x1,y1,z1, s=100, c= "red")
# P2 = ax4.scatter(x2,y2,z2, s=100, c = "grey")
MS = ax4.plot_surface(x[:split], y[:split], z[:split], facecolors=cmap4(norm4(idx[:split])), antialiased=True, shade=False, alpha=.1, zorder=2.5)
MS2 = ax4.plot_surface(x[split:], y[split:], z[split:], facecolors=cmap4(norm4(idx[split:])), antialiased=True, shade=False, alpha=.1)
cb3 = fig.colorbar(cm.ScalarMappable(norm=norm4, cmap=cmap4), ax=ax4, shrink=0.5, orientation="horizontal", pad=-0.05)
cb3.set_label("N element")
ax4.set_title("element spacing")

# Decorate axes and set initial viewing angles
for row in axes:
    for ax in row:
        ax.set_xlabel("x [km]")
        ax.set_ylabel("y [km]")
        ax.set_zlabel("depth [km]")
        ax.view_init(elev=10, azim=-60)
        ax.set_box_aspect([2,2,1])

fig.tight_layout()
plt.show()

In [ ]:
from qdyn import plot_functions as qdyn_plot
from qdyn.utils import post_processing
from qdyn.utils.post_processing.compute_events import compute_events as compute_events

qdyn_plot.timeseries(p.ot, p.ot_vmax)

In [ ]:
model_dict_ =  compute_events(p.set_dict, p.mesh_dict, p.fault, vmax=0.002)

In [ ]:
props = qdyn_plot.plot_events(model_dict_, [1,2])

In [ ]:
import pandas as pd

##############################
## COMBINE OUTPUT SNAPSHOTS ##
##############################

# initialise and merge the two with ox_max
ox_max = p.ox

# if there are overlapping indices, sum the slip
ox_max_grouped = ox_max.groupby(['t', 'x', 'z'], as_index=False)['slip'].sum()

# Merge the summed 'v' values back to the original DataFrame
ox_max_final = pd.merge(ox_max_grouped, ox_max.drop(columns=['slip']), on=['t', 'x', 'z'], how='left')

# Drop duplicate rows to keep only one row per 't', 'x', 'z' combination
ox_max_final = ox_max_final.sort_values(by=['t', 'x', 'z', 'v'], ascending=[True, True, True, False])
ox_max_final = ox_max_final.drop_duplicates(subset=['t', 'x', 'z'])

In [ ]:
qdyn_plot.slip_profile(ox_max_final[(ox_max_final["z"] == -0.445606126784E+04)], warm_up=0, orientation="horizontal")

In [ ]:
ax = qdyn_plot.timestep_profile_all(ox_max_final[(ox_max_final["z"] == -0.445606126784E+04)], p.set_dict["DIP_W"], 'v', warm_up=0, orientation="horizontal")

In [ ]:
qdyn_plot.cv_plot(model_dict_)

In [ ]:
ax = qdyn_plot.fault_progression_plot(p, xoffset = xoffset, vmax = 1e-4, show_nucleations=True, time=[0,2000])